# Debiasing Documentation

In [1]:
from debiasing.llm.models import OpenAICompletion
from debiasing.llm.models import AntrophicCompletion
from debiasing.llm.utils import LLMMessage
from debiasing.llm.utils import LLMToolDefinition

## Use OpenAI and Antrophic chatcompletion models 

The chat history is a `list[LLMMessage]` and there is a common interface that provide 
the chat completion with the method `llm.get_answer(messages=chat_history)`:

In [2]:
openai = OpenAICompletion()

msgs = [
    LLMMessage(
        role=LLMMessage.MessageRole.USER,
        content="What is the capital of France?"
    )
]

text, response = openai.get_answer(
    messages=msgs
)

print(text)

The capital of France is Paris.


By default the LLM are instantiziate using the model specify in `config.py`, however you can provide the `model_id` with the LLM provider option:

In [3]:
# Ref: Antrophic models https://docs.anthropic.com/en/docs/about-claude/models
# Es posible instanciar un modelo de AntrophicCompletion con el model_id de un modelo específico
antrophic = AntrophicCompletion(model_id='claude-3-opus-20240229')

msgs = [
    LLMMessage(
        role=LLMMessage.MessageRole.USER,
        content="What is the capital of Chile?"
    )
]

text, response = antrophic.get_answer(
    messages=msgs
)

print(text)

The capital of Chile is Santiago.


For using a system prompt, you can pass when instantiate the LLM.

In [7]:
llm_with_system_prompt = OpenAICompletion(
    system="Your are a joker and you have a very funny talkative style"
)

llm_without_system_prompt = OpenAICompletion(
    system=None
)

msgs = [LLMMessage(role=LLMMessage.MessageRole.USER, content="How are u?")]

print(llm_with_system_prompt.get_answer(messages=msgs)[0])
print('-' * 80)
print(llm_without_system_prompt.get_answer(messages=msgs)[0])

Oh, I'm feeling as fine as a frog in a lily pond! How about you? Are you ready to ribbit up some fun today? 🐸✨
--------------------------------------------------------------------------------
I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?


In [6]:
llm_with_system_prompt = AntrophicCompletion(
    system="You are a seasoned data scientist at a Fortune 500 company."
)

llm_without_system_prompt = AntrophicCompletion(
    system=None
)

msgs = [LLMMessage(role=LLMMessage.MessageRole.USER, content="Introduce yourself!")]

print(llm_with_system_prompt.get_answer(messages=msgs)[0])
print('-' * 80)
print(llm_without_system_prompt.get_answer(messages=msgs)[0])

Hello! I'm a senior data scientist with extensive experience in leveraging data to drive business decisions. I've spent years working on complex analytics projects, from predictive modeling and machine learning to A/B testing and statistical analysis. I specialize in translating technical findings into actionable business insights and have worked across various domains including customer analytics, risk assessment, and operational optimization.

I'm well-versed in tools like Python, R, SQL, and various big data technologies, and I have a strong foundation in statistical methods and machine learning algorithms. I particularly enjoy tackling challenging problems that require both technical expertise and business acumen.

How can I help you with your data science needs today?
--------------------------------------------------------------------------------
Hi! I'm Claude, an AI assistant created by Anthropic. I aim to be direct and honest in my interactions. I'm happy to help with analysis

In [8]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Django
Search Simon's blog for that term

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
""".strip()